Install dependencies

In [ ]:
%pip install torch torchvision torchaudio
%pip install jupyter
%pip install ipywidgets widgetsnbextension pandas-profiling

Generate 10,000 examples of age-label pairs, where the label is the same as the age. We save this data to a file called "age_data.txt".

In [ ]:
import random
import torch

# Generate synthetic data
data = []
for i in range(10000):
    age = random.randint(0, 100)
    age = torch.tensor(age, dtype=torch.float32)
    
    age = age / 100
    data.append((age, age))

# Save data to a file
with open("age_data.txt", "w") as f:
    for age, label in data:
        f.write(f"{age}\t{label}\n")


Define pytorch model and the predict_age function

In [ ]:
import torch.nn as nn


class AgeModel(nn.Module):
    def __init__(self):
        super(AgeModel, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        # this model has the following structure:
        # input -> one neuron -> output

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        return x
    

def predict_age(model, age):
    with torch.no_grad():
        x = torch.tensor([age]).reshape(-1, 1)
        y_pred = model(x)
        return y_pred.item()


This model consists of two linear layers with ReLU activation in between. We will use mean squared error (MSE) as our loss function and stochastic gradient descent (SGD) as our optimizer.

Train the model

In [ ]:
import torch.optim as optim

# Define hyperparameters
learning_rate = 0.01
batch_size = 64
num_epochs = 100

# Load data
data = []
with open("age_data.txt", "r") as f:
    for line in f:
        age, label = line.strip().split("\t")
        data.append((float(age), float(label)))

# Split data into training and validation sets
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

# Define model, loss function, and optimizer
model = AgeModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    # Shuffle training data
    random.shuffle(train_data)

    # Train on batches
    for i in range(0, len(train_data), batch_size):
        batch = train_data[i:i+batch_size]
        x = torch.tensor([d[0] for d in batch]).reshape(-1, 1)
        y = torch.tensor([d[1] for d in batch]).reshape(-1, 1)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

    # Compute validation loss
    val_loss = 0
    with torch.no_grad():
        for i in range(0, len(val_data), batch_size):
            batch = val_data[i:i+batch_size]
            x = torch.tensor([d[0] for d in batch]).reshape(-1, 1)
            y = torch.tensor([d[1] for d in batch]).reshape(-1, 1)

            outputs = model(x)
            val_loss += criterion(outputs, y).item()

    # Print training and validation loss for the current epoch
    epoch += 1
    if epoch % 10 == 0:
        print(
            f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Val Loss = {val_loss/len(val_data):.4f}")


Predict the age!

In [ ]:
# Predict age based on user input
age = float(input("Enter your age: "))
age = age / 100
predicted_age = predict_age(model, age)
predicted_age = predicted_age * 100
print(f"Predicted age: {predicted_age:.0f}")
